## CS3110 Final Project

Jack Giuliani & Alex Shapovalov

Our project uses data from “SyntheticHealth” which is an organization that creates mock patient data for ML use. We used a variety of methods to analyze this data, including applying Laplace and Gaussian noise, k-anonymizing data, and clipping. We also tested the Propose-Test-Release and Sample and Aggregate frameworks as methods to find average and maximum values for some numeric categories while still achieving differential privacy.

In [20]:
# Base code from class

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def is_k_anonymous(k, qis, df):
    return df[qis].value_counts().min() >= k

# The data: https://synthetichealth.github.io/synthea/

data = pd.read_csv('patients.csv')

print("First 5 rows: ")
print(data.head(5))

First 5 rows: 
                                     Id   BIRTHDATE DEATHDATE          SSN  \
0  b9c610cd-28a6-4636-ccb6-c7a0d2a4cb85  2019-02-17       NaN  999-65-3251   
1  c1f1fcaa-82fd-d5b7-3544-c8f9708b06a8  2005-07-04       NaN  999-49-3323   
2  339144f8-50e1-633e-a013-f361391c4cff  1998-05-11       NaN  999-10-8743   
3  d488232e-bf14-4bed-08c0-a82f34b6a197  2003-01-28       NaN  999-56-6057   
4  217f95a3-4e10-bd5d-fb67-0cfb5e8ba075  1993-12-23       NaN  999-91-4320   

     DRIVERS    PASSPORT PREFIX       FIRST            LAST SUFFIX  ...  \
0        NaN         NaN    NaN    Damon455      Langosh790    NaN  ...   
1  S99941126         NaN    NaN       Thi53       Wunsch504    NaN  ...   
2  S99996708  X75063318X    Mr.      Chi716  Greenfelder433    NaN  ...   
3  S99929424         NaN    Ms.  Phillis443       Walter473    NaN  ...   
4  S99991143  X44132498X    Mr.  Jerrold404       Herzog843    NaN  ...   

                         BIRTHPLACE                           ADD

In [21]:
# 1. Laplace for healthcare expenses and healhcare coverage here:

def dp_sum_expenses(epsilon):
    # Calculate the dp sum of the expenses column
    b = 5000000
    clip_sum = data['HEALTHCARE_EXPENSES'].clip(lower=0, upper=b).sum()
    noisy_sum = laplace_mech(clip_sum, b, epsilon)
    return noisy_sum

def dp_sum_coverage(epsilon):
    b = 500000
    # Calculate the dp sum of the coverage column
    clip_sum = data['HEALTHCARE_COVERAGE'].clip(lower=0, upper=b).sum()
    noisy_sum = laplace_mech(clip_sum, b, epsilon)
    return noisy_sum

def dp_avg_expenses(epsilon):
    # Calculate the dp average of the expenses column
    noisy_sum = dp_sum_expenses(epsilon/2)
    count = len(data)
    noisy_count = laplace_mech(count, 1, epsilon/2)
    return noisy_sum / noisy_count

def dp_avg_coverage(epsilon):
    # Calculate the dp average of the coverage column
    noisy_sum = dp_sum_coverage(epsilon/2)
    count = len(data)
    noisy_count = laplace_mech(count, 1, epsilon/2)
    return noisy_sum / noisy_count

laplace_expenses = dp_avg_expenses(1)
laplace_coverage = dp_avg_coverage(1)
print(f"The average laplace dp expenses is: ${laplace_expenses:.2f}")
print(f"The average laplace dp coverage is: ${laplace_coverage:.2f}")

The average laplace dp expenses is: $1226254.65
The average laplace dp coverage is: $125918.69


In [22]:
# 2. Guassian for healthcare expenses and healthcare coverage here:

def dp_sum_expenses(epsilon, delta):
    # Calculate the dp sum of the expenses column
    b = 5000000
    clip_sum = data['HEALTHCARE_EXPENSES'].clip(lower=0, upper=b).sum()
    noisy_sum = gaussian_mech(clip_sum, b, epsilon, delta)
    return noisy_sum

def dp_sum_coverage(epsilon, delta):
    # Calculate the dp sum of the coverage column
    b = 500000
    clip_sum = data['HEALTHCARE_COVERAGE'].clip(lower=0, upper=b).sum()
    noisy_sum = gaussian_mech(clip_sum, b, epsilon, delta)
    return noisy_sum

def dp_avg_expenses(epsilon, delta):
    # Calculate the dp average of the expenses column
    noisy_sum = dp_sum_expenses(epsilon/2, delta)
    count = len(data)
    noisy_count = gaussian_mech(count, 1, epsilon/2, delta)
    return noisy_sum / noisy_count

def dp_avg_coverage(epsilon, delta):
    # Calculate the dp average of the coverage column
    noisy_sum = dp_sum_coverage(epsilon/2, delta)
    count = len(data)
    noisy_count = gaussian_mech(count, 1, epsilon/2, delta)
    return noisy_sum / noisy_count

gaussian_expenses = dp_avg_expenses(1, 1e-5)
gaussian_coverage = dp_avg_coverage(1, 1e-5)
print(f"The average gaussian dp expenses is: ${gaussian_expenses:.2f}")
print(f"The average gaussian dp coverage is: ${gaussian_coverage:.2f}")

The average gaussian dp expenses is: $1173180.50
The average gaussian dp coverage is: $123075.97


In [23]:
# 3. Comparison between laplace and gaussian

def real_average_expenses():
    return data['HEALTHCARE_EXPENSES'].mean()

def real_average_coverage():
    return data['HEALTHCARE_COVERAGE'].mean()

def percent_difference(real, estimated):
    return abs(real - estimated) / real * 100

real_expenses = real_average_expenses()
real_coverage = real_average_coverage()

percent_diff_laplace_expenses = percent_difference(real_expenses, laplace_expenses)
percent_diff_laplace_coverage = percent_difference(real_coverage, laplace_coverage)
percent_diff_gaussian_expenses = percent_difference(real_expenses, gaussian_expenses)
percent_diff_gaussian_coverage = percent_difference(real_coverage, gaussian_coverage)

print(f"Percent diff laplace expenses: {percent_diff_laplace_expenses:.2f}%")
print(f"Percent diff laplace coverage: {percent_diff_laplace_coverage:.2f}%")
print(f"Percent diff gaussian expenses: {percent_diff_gaussian_expenses:.2f}%")
print(f"Percent diff gaussian coverage: {percent_diff_gaussian_coverage:.2f}%")

Percent diff laplace expenses: 2.39%
Percent diff laplace coverage: 26.69%
Percent diff gaussian expenses: 6.61%
Percent diff gaussian coverage: 28.35%


In [24]:
# 4. Create k-anonymous address data here

def anonymize(data, k):
    # Group by CITY and STATE, filter groups with fewer than k records
    group = data.groupby(['CITY', 'STATE'])
    filtered = group.filter(lambda x: len(x) >= k)

    # Generalize coordinate information, remove specific addresses
    filtered['LAT'] = filtered['LAT'].round(1)
    filtered['LON'] = filtered['LON'].round(1)
    filtered['ADDRESS'] = 'REDACTED'
    
    return filtered

k = 5
data_anonymous = anonymize(data, k)

# Print first 10 and last 10 addresses
print("Anonymized Data:")
print(data_anonymous[['ADDRESS', 'CITY', 'STATE', 'LAT', 'LON']].head(10))
print(data_anonymous[['ADDRESS', 'CITY', 'STATE', 'LAT', 'LON']].tail(10))

Anonymized Data:
     ADDRESS         CITY          STATE   LAT   LON
0   REDACTED  Springfield  Massachusetts  42.1 -72.5
2   REDACTED       Boston  Massachusetts  42.3 -71.1
4   REDACTED       Revere  Massachusetts  42.4 -71.0
6   REDACTED       Revere  Massachusetts  42.4 -71.0
7   REDACTED   Somerville  Massachusetts  42.4 -71.1
8   REDACTED    Cambridge  Massachusetts  42.3 -71.1
9   REDACTED  Springfield  Massachusetts  42.1 -72.5
11  REDACTED   Somerville  Massachusetts  42.4 -71.1
12  REDACTED    Wellesley  Massachusetts  42.3 -71.3
13  REDACTED       Boston  Massachusetts  42.4 -71.1
       ADDRESS           CITY          STATE   LAT   LON
1148  REDACTED         Boston  Massachusetts  42.4 -71.0
1149  REDACTED        Peabody  Massachusetts  42.6 -71.0
1151  REDACTED  Middleborough  Massachusetts  41.8 -70.8
1152  REDACTED       Scituate  Massachusetts  42.2 -70.7
1153  REDACTED     Framingham  Massachusetts  42.3 -71.4
1157  REDACTED     Marblehead  Massachusetts  42.5 -70.9
1

In [25]:
# 5. Perform a linkage attack

In [26]:
data_pii = data[['FIRST', 'LAST', 'BIRTHDATE', 'CITY', 'STATE', 'ZIP']]
data_pii.head()

,FIRST,LAST,BIRTHDATE,CITY,STATE,ZIP
0,Damon455,Langosh790,2019-02-17,Springfield,Massachusetts,1104.0
1,Thi53,Wunsch504,2005-07-04,Bellingham,Massachusetts,NaN
2,Chi716,Greenfelder433,1998-05-11,Boston,Massachusetts,2131.0
3,Phillis443,Walter473,2003-01-28,Hingham,Massachusetts,2043.0
4,Jerrold404,Herzog843,1993-12-23,Revere,Massachusetts,NaN


In [27]:
data_deid = data.drop(columns=['Id', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'BIRTHPLACE', 'ADDRESS', 'LAT', 'LON'])
data_deid.head()

,BIRTHDATE,MARITAL,RACE,ETHNICITY,GENDER,CITY,STATE,COUNTY,ZIP,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,2019-02-17,NaN,white,nonhispanic,M,Springfield,Massachusetts,Hampden County,1104.0,9039.1645,7964.1255
1,2005-07-04,NaN,white,nonhispanic,F,Bellingham,Massachusetts,Norfolk County,NaN,402723.4150,14064.1350
2,1998-05-11,NaN,white,nonhispanic,M,Boston,Massachusetts,Suffolk County,2131.0,571935.8725,787.5375
3,2003-01-28,NaN,white,nonhispanic,F,Hingham,Massachusetts,Plymouth County,2043.0,582557.8030,104782.2070
4,1993-12-23,M,black,nonhispanic,M,Revere,Massachusetts,Suffolk County,NaN,475826.8550,18067.0950


In [28]:
def linking_attack(cols):
    data_linked = pd.merge(data_pii, data_deid, left_on=cols, right_on=cols)
    return data_linked

print(len(linking_attack(['ZIP'])))
print(len(linking_attack(['BIRTHDATE'])))
print(len(linking_attack(['ZIP', 'BIRTHDATE'])))
print(len(linking_attack(['ZIP', 'BIRTHDATE', 'CITY'])))

299659
1851
1533
1531


In [29]:
# 6. Clipping on expenses column

In [30]:
# TODO find way to automatically select clipping parameter b
def find_b():
    bs = range(1, 5000000, 100000)
    epsilon_i = 1/len(bs)
    for b in bs:
        result = data['HEALTHCARE_EXPENSES'].clip(upper=b).sum()
        dp_result = laplace_mech(result, sensitivity=b, epsilon=epsilon_i)
        print(b, dp_result)

def find_b_exp():
    bs = [2**x for x in list(range(1, 26))]
    epsilon_i = 1/len(bs)
    for b in bs:
        result = data['HEALTHCARE_EXPENSES'].clip(upper=b).sum()
        dp_result = laplace_mech(result, sensitivity=b, epsilon=epsilon_i)
        print(b, dp_result)
        
find_b_exp()

2 2281.1018241717925
4 4731.7271698252425
8 9349.639898264573
16 17937.599077407627
32 36414.28685333451
64 75131.44145293183
128 149340.98883396373
256 299272.23306028766
512 609141.2531523996
1024 1203625.858056843
2048 2402970.7105649975
4096 4601781.143014051
8192 9893846.300425934
16384 17949876.17923666
32768 38060592.73075961
65536 74423045.2952968
131072 143169822.01451728
262144 271312671.3447172
524288 510129431.40824676
1048576 911315926.2194884
2097152 1422237471.205084
4194304 1521220855.6136854
8388608 1368851584.7319727
16777216 1503159162.259734
33554432 1402226750.906887


In [44]:
def mean_expenses_clipping(epsilon):
    df = data['HEALTHCARE_EXPENSES']
    lower = 0
    upper = 25000000

    dp_sum = laplace_mech(df.clip(upper=upper).sum(), sensitivity=upper, epsilon=epsilon/2)
    dp_count = laplace_mech(len(df), sensitivity=1, epsilon=epsilon/2)
    return dp_sum/dp_count


test_results = [mean_expenses_clipping(1.0) for i in range(100)]
average_result = sum(test_results)/len(test_results)
print("Average expenses with clipping:", average_result)
print("Percent difference:", percent_difference(data['HEALTHCARE_EXPENSES'].mean(), average_result))

Average expenses with clipping: 1264450.9793286968
Percent difference: 0.6529374563802481


In [32]:
# 7. Propose test release vs SAA for average

In [46]:
# TODO fix with better value for s
def ls_mean_at_distance(df, b, k):
    n = len(df)
    return b/(n - k)

def dist_to_high_ls_mean(df, b, s):
    k = 0
    while ls_mean_at_distance(df, b, k) < s:
        k += 1
    return k

def mean_expenses_ptr(s, epsilon, delta):
    df = data['HEALTHCARE_EXPENSES']
    lower = 0
    upper = 25000000
    
    distance = dist_to_high_ls_mean(df, upper, s)
    noisy_distance = laplace_mech(distance, sensitivity=1, epsilon=epsilon)
    test_result = noisy_distance < np.log(2/delta)/(2*epsilon)
    if test_result:
        return False

    result = df.clip(lower=0, upper=upper).mean()
    return laplace_mech(result, sensitivity=s, epsilon=epsilon)

test_results = [mean_expenses_ptr(10000, 1, 1e-5) for i in range(100)]
average_result = sum(test_results)/len(test_results)
print("Average expenses with PTR:", average_result)
print("Percent difference:", percent_difference(data['HEALTHCARE_EXPENSES'].mean(), average_result))

Average expenses with PTR: 12466.651589646093
Percent difference: 99.00762850971945


In [34]:
def f(chunk):
    return chunk.mean()

def mean_expenses_saa(k, epsilon):
    df = data['HEALTHCARE_EXPENSES']
    lower = 0
    upper = 2900001
    
    chunks = np.array_split(df, k)
    answers = [f(chunk) for chunk in chunks]
    answers_clipped = pd.Series(answers).clip(lower=lower, upper=upper)
    answers_clipped_avg = answers_clipped.mean()
    return laplace_mech(answers_clipped_avg, sensitivity=upper/k, epsilon=epsilon)

test_result = [mean_expenses_saa(20, 1.0) for i in range(100)]
average_result = sum(test_results)/len(test_results)
print("Average expenses with SAA:", average_result)
print("Percent difference:", percent_difference(data['HEALTHCARE_EXPENSES'].mean(), average_result))

Average expenses with SAA: 1216527.9622836728
Percent difference: 3.1618347382007723


In [35]:
# 8. Propose test release vs SAA for maximum

In [36]:
def ls_max_at_distance(k, sorted_expenses, expenses_lower):
    if k < len(sorted_expenses):
        return sorted_expenses.iloc[- (k+ 1)]
    else:
        return expenses_lower

def dist_to_low_ls_max(s_p, sorted_expenses, lower):
    k = 0
    while ls_max_at_distance(k, sorted_expenses, lower) > s_p:
        k += 1
    return k
    
def max_expenses_ptr(s_p, epsilon, delta):
    df = data['HEALTHCARE_EXPENSES']
    lower = 0
    upper = 25000000
    sorted_expenses = df.clip(lower=lower, upper=upper).sort_values()
    distance = dist_to_low_ls_max(s_p, sorted_expenses, lower)
    noisy_distance = laplace_mech(distance, sensitivity=1, epsilon=epsilon)
    test_result = noisy_distance < np.log(2/delta)/(2*epsilon)
    if test_result:
        return None
        
    result = df.clip(lower=lower, upper=upper).max()
    return laplace_mech(result, sensitivity=s_p, epsilon=epsilon)
    
test_results = [max_expenses_ptr(10000, 1, 1e-5) for i in range(100)]
average_result = sum(test_results)/len(test_results)
print("Max expenses with PTR:", average_result)
print("Percent difference:", percent_difference(data['HEALTHCARE_EXPENSES'].max(), average_result))

Max expenses with PTR: 23116821.107847948
Percent difference: 0.0011755597417354112


In [42]:
def f(chunk):
    return chunk.max()

def max_expenses_saa(k, epsilon):
    df = data['HEALTHCARE_EXPENSES']
    lower = 0
    upper = 25000000
    
    chunks = np.array_split(df, k)
    answers = [f(chunk) for chunk in chunks]
    answers_clipped = pd.Series(answers).clip(lower=lower, upper=upper)
    answers_avg = answers_clipped.mean()
    return laplace_mech(answers_avg, sensitivity=upper/k, epsilon=epsilon)

test_results = [max_expenses_saa(20, 1.0) for i in range(100)]
average_result = sum(test_results)/len(test_results)
print("Max expenses with SAA:", average_result)
print("Percent difference:", percent_difference(data['HEALTHCARE_EXPENSES'].max(), average_result))

Max expenses with SAA: 4849621.509347453
Percent difference: 79.02099732086812
